In [1]:
from noid_wrapper import noid_client
client = noid_client.NoidClient("/home/srappel/noid_wrapper/config.yaml")

In [2]:
from pathlib import Path
import json

def process_metadata_files(base_dir):
    """Recursively processes metadata files in directories to extract ARK IDs."""
    documents = []
    base_path = Path(base_dir)
    
    for json_file in base_path.rglob("*.json"):
        with json_file.open('r') as file:
            data = json.load(file)
            ark_id = data.get("dct_identifier_sm")
            if ark_id:
                documents.append((ark_id, data))
    
    return documents


In [ ]:
target_dir = Path("/home/srappel/noid_wrapper/metadata")
target_documents = process_metadata_files(target_dir)

i = 0
for documnet in target_documents:
    i += 1
    print(i, documnet[0])
    


In [ ]:
rejects = []
for document in target_documents:
    noid_id = document[0][0].removeprefix("ark:/")
    
    # Validate the ID
    if not client.validate(noid_id):
        rejects.append(noid_id)
        continue

    # Make a dict of the elements to be bound...
    where_part = noid_id.replace("/","-")
    where = f"https://geodiscovery.uwm.edu/catalog/ark:-{where_part}"
    title = document[1].get("dct_title_s", "Untitled")
    permalink = f"https://digilib.uwm.edu/ark:/{noid_id}"

    bind_params = {
        "where": where,
        "title": title,
        "permalink": permalink
    }

    client._run_noid_command("hold", "set", noid_id)

    client.bind_multiple(noid_id, bind_params)

    

        
